In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


***Cleaning*

In [4]:
def lowercase(data):
    result = ""
    for i in data:
        if i.isupper():
            result += i.lower()
        else:
            result += i
    return result

print(lowercase("ManpreetS"))


manpreets


In [5]:
df["review"]=df["review"].apply(lowercase)

In [6]:
df['review'] = df['review'].str.replace(r'<.*?>','')
df['review'] = df['review'].str.replace(r"https?://\S+|www\.\S+",'')

In [7]:
import re
def remove_url(data):
    data=re.sub(r"https?://\S+|www\.\S+",'',data)
    data=re.sub(r'<.*?>','',data)
    return data


In [8]:
df["review"]=df["review"].apply(remove_url)

In [9]:
import string

def remove_pun(text):
    for i in string.punctuation:
        if i in text:
            text = text.replace(i, " ")
    return text
df["review"]=df["review"].apply(remove_pun)

In [10]:
bad_chars = [';', ':', '!', "*", "^","&","(",")","$","[","]"]

def remove_sym(text):
    test_string = text
    for char in bad_chars:
        test_string = test_string.replace(char, '')
    return test_string
 
df["review"]=df["review"].apply(remove_sym)

In [11]:
def replace_multiple_spaces(text):
    
    return re.sub(r'\s+', ' ', text)
df["review"]=df["review"].apply(replace_multiple_spaces)

In [12]:
print(df.review[0])

one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn t dare forget pretty 

In [13]:
df.to_csv("pd2.csv")

In [14]:
from nltk.tokenize import word_tokenize
df['tokenized_review'] = df['review'].apply(word_tokenize)

In [15]:
from nltk.corpus import stopwords
def remove_stopwords(text):

    L = []
    for word in text:
        if word not in stopwords.words('english'):
            L.append(word)

    return L

In [16]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
y=[]
def stem_words(text):
    for i in text:
        y.append(ps.stem(i))
    z=y[:]
    y.clear()
    return z
        

In [17]:
df['tokenized_review'] = df['tokenized_review'].apply(remove_stopwords)
df['tokenized_review'] = df['tokenized_review'].apply(stem_words)

In [18]:
print(df.tokenized_review[0])

['one', 'review', 'mention', 'watch', '1', 'oz', 'episod', 'hook', 'right', 'exactli', 'happen', 'first', 'thing', 'struck', 'oz', 'brutal', 'unflinch', 'scene', 'violenc', 'set', 'right', 'word', 'go', 'trust', 'show', 'faint', 'heart', 'timid', 'show', 'pull', 'punch', 'regard', 'drug', 'sex', 'violenc', 'hardcor', 'classic', 'use', 'word', 'call', 'oz', 'nicknam', 'given', 'oswald', 'maximum', 'secur', 'state', 'penitentari', 'focus', 'mainli', 'emerald', 'citi', 'experiment', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inward', 'privaci', 'high', 'agenda', 'em', 'citi', 'home', 'mani', 'aryan', 'muslim', 'gangsta', 'latino', 'christian', 'italian', 'irish', 'scuffl', 'death', 'stare', 'dodgi', 'deal', 'shadi', 'agreement', 'never', 'far', 'away', 'would', 'say', 'main', 'appeal', 'show', 'due', 'fact', 'goe', 'show', 'dare', 'forget', 'pretti', 'pictur', 'paint', 'mainstream', 'audienc', 'forget', 'charm', 'forget', 'romanc', 'oz', 'mess', 'around', 'first', 'episod', '

In [19]:
def join_back(list_input):
    return " ".join(list_input)
    
df['review']=df['tokenized_review'].apply(join_back)

In [20]:
X=df["review"].values

y=df["sentiment"].values
print(y)

['positive' 'positive' 'positive' ... 'negative' 'negative' 'negative']


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500)
X=cv.fit_transform(df['review']).toarray()

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [23]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
clf1=GaussianNB()
clf2=MultinomialNB()
clf3=BernoulliNB()
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf3.predict(X_test)

from sklearn.metrics import accuracy_score
print("Gaussian",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernaulli",accuracy_score(y_test,y_pred3))

Gaussian 0.7512
Multinomial 0.8398
Bernaulli 0.843


In [24]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [25]:
y_pred4=neigh.predict(X_test)
from sklearn.metrics import accuracy_score
print("knn",accuracy_score(y_test,y_pred4))

knn 0.6325
